<svg width="300" height="100" xmlns="http://www.w3.org/2000/svg">
<line x1="10" y1="10" x2="290" y2="10" style="stroke:#000">
</line>
<ellipse cx="150" cy="50" rx="20" ry="20" style="stroke:#000;fill:none;"></ellipse>
</svg>

In [ ]:
from IPython.display import SVG
import svgwrite
import numpy as np
import os
import random
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
import warnings
import cmath
from svgpathtools import *
import math

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
paths, attributes, svg_attributes = svg2paths2('Minisample/leaves.svg')

In [ ]:
n_seg = 80
ts = np.array(range(n_seg+1))*1/n_seg

In [ ]:
figures = []
for path in paths:
    figures.append([path.point(t) for t in ts])

In [ ]:
def get_angle(point_t,point_t2):
    diff = point_t2-point_t
    #angle = math.atan(abs(diff.imag)/abs(diff.real))eeerrr
    if diff.real < 0:
        if diff.imag <=0:
            angle = math.pi - math.atan(abs(diff.imag)/abs(diff.real))
        else:
            angle = math.pi + math.atan(abs(diff.imag)/abs(diff.real))
    else:
        if diff.imag > 0:
            angle = 2*math.pi - math.atan(abs(diff.imag)/abs(diff.real))
        else:
            angle = math.atan(abs(diff.imag)/abs(diff.real))
    if angle < 0:
        print('angle generation error')
    return angle        

In [ ]:
# We shall try out real and delta angles 
def get_coord(angle,hyp):
    if angle <0:
        angle = 2*math.pi + angle
    if angle > math.pi/2:
        if angle >= 2*math.pi:
            angle = angle - 2*math.pi
        if angle <= math.pi:
            temp_angle = math.pi - angle
            x_delta = -math.cos(temp_angle)*hyp
            y_delta = -math.sin(temp_angle)*hyp
        elif angle <= 1.5*math.pi:
            temp_angle = angle - math.pi
            x_delta = -math.cos(temp_angle)*hyp
            y_delta = math.sin(temp_angle)*hyp
        elif angle <= 2*math.pi:
            temp_angle = 2*math.pi - angle
            x_delta = math.cos(temp_angle)*hyp
            y_delta = math.sin(temp_angle)*hyp
    else:
        x_delta = math.cos(angle)*hyp
        y_delta = -math.sin(angle)*hyp  
    return (x_delta,y_delta)

In [ ]:
training_data = []
for points in figures:
    angles = [0]
    for i in range(1,n_seg+1):
        angles.append(get_angle(points[i-1],points[i]))
    angles = np.array(angles)[1:]-np.array(angles)[:-1]
    training_data.append(angles)
training_data = torch.unsqueeze(torch.tensor(training_data,dtype=torch.float,device=cuda0),2)

In [ ]:
points = figures[2]

##### Idea: first convert to the line format according to the necessary #of points and then split evenly (smallest element)
##### Hierarchical generation - first generate the structure and then indicidual item
##### conceptual learning

In [ ]:
segments =[Line(points[0],points[1])]
real_segments = []
angles = []
for i in range(1,n_seg+1):
    angle = get_angle(points[i-1],points[i])
    diff = points[i] - points[i-1]
    #print(math.sqrt(math.pow(diff.real,2)+math.pow(diff.imag,2)))
    angles.append(angle)
    x_delta,y_delta = get_coord(angle,1)
    shift = complex(20,0)
    real_segments.append(Line(points[i-1] + shift,points[i] + shift))
    segments.append(Line(segments[-1].end,segments[-1].end+complex(x_delta,y_delta)))
wsvg(segments +real_segments, filename='test_single_model.svg')
SVG('test_single_model.svg')

In [ ]:
x = training_data[:,:-1,:]
y = training_data[:,1:,:]

In [ ]:
y[4,0,0]

In [ ]:
x_batches = torch.split(x,50)

In [ ]:
y_batches = torch.split(y,50)

In [ ]:
class LSTMArtist(nn.Module):

    def __init__(self, input_dim, hidden_dim, mix_k, num_layers=2,dropout=0.1): #, dropout=0.1
        super(LSTMArtist, self).__init__()
        
        #self.cov = torch.eye(6)
        #self.cov.requires_grad = True
        
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers=num_layers, dropout=dropout, batch_first=True)
        
        self.linear_mu = nn.Linear(hidden_dim,mix_k)
        self.linear_sigmasq = nn.Linear(hidden_dim,1)
        self.linear_mix = nn.Linear(hidden_dim,mix_k)
        
        self.activation = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=2)
        self.relu = nn.ReLU()


    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        coordinates_mu = self.linear_mu(lstm_out)
        coordinates_sigmasq = torch.exp(self.linear_sigmasq(lstm_out)) #torch.exp self.relu
        coordinates_mix = self.softmax(self.linear_mix(lstm_out))
        return (coordinates_mu,coordinates_sigmasq,coordinates_mix)

In [ ]:
def gaussian_pdf(x, mu, sigmasq):
    # NOTE: we could use the new `torch.distributions` package for this now
    #print((-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2)
    #print(torch.exp((-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2))
    pdf = (1/torch.sqrt(2*np.pi*sigmasq)) * torch.exp(torch.min(torch.tensor(50).to(cuda0),torch.max(torch.tensor(-50).to(cuda0),(-1/(2*sigmasq))* torch.norm((x-mu), 2, 2)**2)))
    return pdf

In [ ]:
def calculate_loss(coordinates_mu,coordinates_sigmasq,coordinates_mix,k,y):
    losses = torch.zeros(y.shape[0:2],device = cuda0)
    for i in range(k):  # To Do: put k versions for method and absolute, add loss proportionally likelihood
        likelihood_z_x = gaussian_pdf(y, coordinates_mu[:,:,i:(i+1)], coordinates_sigmasq[:,:, 0])
        #print(likelihood_z_x)
        losses += coordinates_mix[:,:,i] * likelihood_z_x
    return torch.mean(-(torch.log(losses)))

In [ ]:
k = 20
model = LSTMArtist(1,16,k,dropout=0.2,num_layers=2).to(cuda0) #.to(cuda0) ,dropout=0.9
##################
#mse_loss = nn.MSELoss()
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
#################
for epoch in range(20000):
    if epoch % 1000 == 0:
        print("Running epoch",epoch)
    total_loss = 0
    for i,batch in enumerate(x_batches):
        optimizer.zero_grad()
        coordinates_mu,coordinates_sigmasq,coordinates_mix = model(batch)
        #print(coordinates_sigmasq)
        loss = calculate_loss(coordinates_mu,coordinates_sigmasq,coordinates_mix,k,y_batches[i])
        #print(">>>>>>>>>>>>>>>>>>>",losses)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer.step()
        #print(output.size())
        total_loss += loss.item()
    if epoch % 1000 == 0:
        print(total_loss) #with 10 epochs 7720 2.36

In [ ]:
torch.save(model.state_dict(), 'model_MDM_MULti.model')

In [ ]:
def random_sample(coordinates_mu,coordinates_sigmasq,coordinates_mix):
    with torch.no_grad():
        #coordinates_mu = coordinates_mu.cpu()
        #coordinates_sigmasq = coordinates_sigmasq.cpu()
        #coordinates_mix = coordinates_mix.cpu()

        mix_dist = Categorical(coordinates_mix[0,0,:])
        mix_index = mix_dist.sample()
        mu = coordinates_mu[0,0,mix_index:(mix_index+1)]
        sigma = torch.sqrt(coordinates_sigmasq)
        sigma = torch.tensor([[[0.00001]]],device=cuda0)
        angle = Normal(mu, sigma).sample()
        #print(str_key,absolute,coordinates)
        return angle

In [ ]:
#vectors = start_vector        
coordinates_mu,coordinates_sigmasq,coordinates_mix = model(vectors)
angle = random_sample(coordinates_mu[:,-1:,:],coordinates_sigmasq[:,-1:,:],coordinates_mix[:,-1:,:])
vectors = torch.cat((vectors,angle),1)

In [ ]:
# Generate the picture from the trained model
def generate_vectors(model,start_vector,iterations):
    vectors = start_vector
    for i in range(iterations):
        coordinates_mu,coordinates_sigmasq,coordinates_mix = model(vectors)
        angle = random_sample(coordinates_mu[:,-1:,:],coordinates_sigmasq[:,-1:,:],coordinates_mix[:,-1:,:])
        vectors = torch.cat((vectors,angle),1)
    return vectors

In [ ]:
start_vector = torch.tensor([[[0.0]]],device=cuda0)
vectors = generate_vectors(model,start_vector,80)
vectors = vectors[0,:,0]
real_angles = [0]
for vector in vectors:
    real_angles.append(real_angles[-1]+vector)

In [ ]:
segments =[]
for i in range(1,len(real_angles)):
    x_delta,y_delta = get_coord(real_angles[i],1)
    if i == 1:
        segments.append(Line(complex(),complex(x_delta,y_delta)))
    else:    
        segments.append(Line(segments[-1].end,segments[-1].end+complex(x_delta,y_delta)))
wsvg(segments, filename='test_algo_2.svg')
SVG('test_algo_2.svg')